In [10]:
import time
import numpy as np
from PIL import Image
from PIL import ImageDraw

import tflite_runtime.interpreter as tflite

from pycoral.adapters import common
from pycoral.adapters import classify
from pycoral.adapters import detect
from pycoral.adapters.detect import BBox
from pycoral.utils.dataset import read_label_file
from pycoral.utils.edgetpu import make_interpreter

from model_runner import ModelRunner

image = Image.open("models/object_detection/data/images/train/2019-04-16-095702.jpg")
labels = read_label_file(
    "models/object_detection/data/model_result/road_sign_labels.txt"
)


def draw_objects(draw, objs, labels):
    """Draws the bounding box and label for each object."""
    for obj in objs:
        bbox = obj.bbox
        draw.rectangle([(bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax)], outline="red")
        draw.text(
            (bbox.xmin + 10, bbox.ymin + 10),
            "%s\n%.2f" % (labels.get(obj.id, obj.id), obj.score),
            fill="red",
        )


In [2]:
image = Image.open("models/object_detection/data/images/train/2019-04-16-095702.jpg")

interpreter = make_interpreter(
    "models/object_detection/data/model_result/road_signs_quantized_edgetpu.tflite"
)
interpreter.allocate_tensors()
_, scale = common.set_resized_input(
      interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

interpreter.invoke()
objs = detect.get_objects(interpreter, 0.3, scale)

for obj in objs:
    print(labels.get(obj.id, obj.id))
    print('  id:    ', obj.id)
    print('  score: ', obj.score)
    print('  bbox:  ', obj.bbox)

image = image.convert("RGB")
draw_objects(ImageDraw.Draw(image), objs, labels)
image.save("models/object_detection/temp.jpg")
image.show()


Red
  id:     2
  score:  0.95703125
  bbox:   BBox(xmin=231, ymin=121, xmax=260, ymax=177)
Limit 25
  id:     3
  score:  0.83984375
  bbox:   BBox(xmin=338, ymin=125, xmax=370, ymax=171)
Red
  id:     2
  score:  0.55859375
  bbox:   BBox(xmin=421, ymin=128, xmax=450, ymax=183)
Stop
  id:     5
  score:  0.5
  bbox:   BBox(xmin=554, ymin=148, xmax=599, ymax=196)
Green
  id:     0
  score:  0.328125
  bbox:   BBox(xmin=18, ymin=133, xmax=52, ymax=197)


In [100]:
org_model_file = [
    "models/object_detection/data/model_result/road_signs_quantized_edgetpu.tflite"
]

# seg_model_files = [
#     "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_0_of_2_edgetpu.tflite",
#     "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_1_of_2_edgetpu.tflite",
# ]
seg_model_files = [
    "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_0_of_3_edgetpu.tflite",
    "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_1_of_3_edgetpu.tflite",
    "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_2_of_3_edgetpu.tflite",
]
# seg_model_files = [
#     "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_0_of_4_edgetpu.tflite",
#     "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_1_of_4_edgetpu.tflite",
#     "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_2_of_4_edgetpu.tflite",
#     "models/object_detection/data/model_result/temp/tmp_models/road_signs_quantized_segment_3_of_4_edgetpu.tflite",
# ]


In [17]:
image = Image.open("models/object_detection/data/images/train/2019-04-16-095702.jpg")

org_model_runner = ModelRunner(org_model_file)
org_model_runner.allocate_tensors_all_interpreter()

st = time.perf_counter()
org_model_runner.invoke_all(task="detection")
objs = org_model_runner.get_objects(score_threshold=0.3)
dur = (time.perf_counter() - st) * 1000
print(dur)

for obj in objs:
    print(labels.get(obj.id, obj.id))
    print('  id:    ', obj.id)
    print('  score: ', obj.score)
    print('  bbox:  ', obj.bbox)

image = image.convert("RGB")
draw_objects(ImageDraw.Draw(image), objs, labels)
image.save("models/object_detection/temp.jpg")
image.show()

8.079741150140762
Red
  id:     2
  score:  0.95703125
  bbox:   BBox(xmin=231, ymin=121, xmax=260, ymax=177)
Limit 25
  id:     3
  score:  0.83984375
  bbox:   BBox(xmin=338, ymin=125, xmax=370, ymax=171)
Red
  id:     2
  score:  0.55859375
  bbox:   BBox(xmin=421, ymin=128, xmax=450, ymax=183)
Stop
  id:     5
  score:  0.5
  bbox:   BBox(xmin=554, ymin=148, xmax=599, ymax=196)
Green
  id:     0
  score:  0.328125
  bbox:   BBox(xmin=18, ymin=133, xmax=52, ymax=197)


In [115]:
image = Image.open("models/object_detection/data/images/train/2019-04-16-095702.jpg")

seg_model_runner = ModelRunner(seg_model_files)
seg_model_runner.allocate_tensors_all_interpreter()

st = time.perf_counter()
seg_model_runner.invoke_all(task="detection")
objs = seg_model_runner.get_objects(score_threshold=0.3)
dur = (time.perf_counter() - st) * 1000
print(dur)

for obj in objs:
    print(labels.get(obj.id, obj.id))
    print('  id:    ', obj.id)
    print('  score: ', obj.score)
    print('  bbox:  ', obj.bbox)

image = image.convert("RGB")
draw_objects(ImageDraw.Draw(image), objs, labels)
image.save("models/object_detection/temp.jpg")
image.show()

---
['FeatureExtractor/MobilenetV2/expanded_conv_16/depthwise/Relu6',
 'BoxPredictor_0/Reshape_1',
 'BoxPredictor_0/Reshape']
---
---
['FeatureExtractor/MobilenetV2/expanded_conv_16/depthwise/Relu6',
 'BoxPredictor_0/Reshape_1',
 'BoxPredictor_0/Reshape',
 'BoxPredictor_1/Reshape_1',
 'BoxPredictor_1/Reshape',
 'FeatureExtractor/MobilenetV2/layer_19_1_Conv2d_2_1x1_256/Relu6']
---
9.457970969378948
Red
  id:     2
  score:  0.95703125
  bbox:   BBox(xmin=231, ymin=121, xmax=260, ymax=177)
Limit 25
  id:     3
  score:  0.83984375
  bbox:   BBox(xmin=338, ymin=125, xmax=370, ymax=171)
Red
  id:     2
  score:  0.55859375
  bbox:   BBox(xmin=421, ymin=128, xmax=450, ymax=183)
Stop
  id:     5
  score:  0.5
  bbox:   BBox(xmin=554, ymin=148, xmax=599, ymax=196)
Green
  id:     0
  score:  0.328125
  bbox:   BBox(xmin=18, ymin=133, xmax=52, ymax=197)
